###Setup
Primeiro, vamos garantir que estamos usando o catálogo correto e que o schema 00_landing existe.

In [0]:
%sql
SELECT current_catalog(), current_schema()

In [0]:
%sql
USE CATALOG smart_claims_dev

In [0]:
%sql
USE SCHEMA `00_landing`

In [0]:
%sql
SELECT current_catalog(), current_schema()

In [0]:
%sql
LIST '/Volumes/smart_claims_dev/00_landing/sql_server'

In [0]:
%python
files= dbutils.fs.ls(
'/Volumes/smart_claims_dev/00_landing/sql_server' 
)
display(files)

In [0]:
%sql
SELECT *
FROM csv.`/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv`
--drop the table if it exists for demonstration

In [0]:
%sql
CREATE TABLE IF NOT EXISTS smart_claims_dev.`01_bronze`.claims
AS SELECT *
FROM read_files('/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
format => 'csv'
)

In [0]:
%sql
SELECT * FROM smart_claims_dev.01_bronze.claims